In [40]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.losses import categorical_crossentropy
from keras.optimizers import adam, sgd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from PIL import Image

In [41]:
#train_path = 'C:/Users/Andy Cai/Desktop/Python/Hack Cambridge/Waste-classification-master/Waste-classification-master/waste-classification-data/DATASET/TRAIN'
#test_path = 'C:/Users/Andy Cai/Desktop/Python/Hack Cambridge/Waste-classification-master/Waste-classification-master/waste-classification-data/DATASET/TEST'
train_path = 'C:/Users/Andy Cai/Desktop/Python/Hack Cambridge/Waste-classification-master/Waste-classification-master/photos/TRAIN'
test_path = 'C:/Users/Andy Cai/Desktop/Python/Hack Cambridge/Waste-classification-master/Waste-classification-master/photos/TEST'
IMG_BREDTH = 30
IMG_HEIGHT = 60
num_classes = 2

In [42]:
train_batch = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=45, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=False).flow_from_directory(train_path, 
                                                                          target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                                          classes=['O', 'R'], 
                                                                          batch_size=10)

test_batch = ImageDataGenerator().flow_from_directory(test_path, 
                                                      target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                      classes=['O', 'R'], 
                                                      batch_size=10)

Found 71 images belonging to 2 classes.
Found 21 images belonging to 2 classes.


In [43]:
def cnn_model():
    
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(IMG_HEIGHT,IMG_BREDTH,3)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
              
    model.summary()
              
    return model

def use_model(path):
    
    model = load_model('model5.h5')
    pic = plt.imread(path)
    pic = cv2.resize(pic, (IMG_BREDTH, IMG_HEIGHT))
    pic = np.expand_dims(pic, axis=0)
    classes = model.predict_classes(pic)
    
#     code using PIL
#     model = load_model('best_waste_classifier.h5')
#     pic1 = plt.imread(path)
#     pic = Image.open(path).resize((IMG_BREDTH, IMG_HEIGHT))
#     plt.imshow(pic1)
#     if model.predict_classes(np.expand_dims(pic, axis=0)) == 0:
#         classes = 'ORGANIC'
#     elif model.predict_classes(np.expand_dims(pic, axis=0)) == 1:
#         classes = 'RECYCLABLE'
    
    return classes

In [44]:
model = cnn_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 60, 30, 32)        896       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 58, 28, 32)        9248      
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 56, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 13, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 28, 13, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 26, 11, 64)        18496     
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 24, 9, 64)         36928     
__________

In [45]:
checkpoint = ModelCheckpoint('model5.h5', 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='auto')

In [46]:
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=1.0e-4), metrics=['accuracy'])

In [47]:
# run code to train the neural network

model = model.fit_generator(train_batch,  
                             validation_data=test_batch,  
                             epochs=500, 
                             verbose=1, 
                             callbacks=[checkpoint])

Epoch 1/500
8/8 [==============================] - 2s 250ms/step - loss: 3.4966 - acc: 0.5690 - val_loss: 2.5970 - val_acc: 0.3333
Epoch 2/500
8/8 [==============================] - 0s 23ms/step - loss: 3.1922 - acc: 0.5690 - val_loss: 0.6441 - val_acc: 0.6667
Epoch 3/500
8/8 [==============================] - 0s 33ms/step - loss: 2.4505 - acc: 0.5944 - val_loss: 0.6880 - val_acc: 0.6667
Epoch 4/500
8/8 [==============================] - 0s 39ms/step - loss: 1.0493 - acc: 0.7338 - val_loss: 0.6838 - val_acc: 0.6190
Epoch 5/500
8/8 [==============================] - 0s 36ms/step - loss: 1.2381 - acc: 0.4690 - val_loss: 0.7190 - val_acc: 0.6667
Epoch 6/500
8/8 [==============================] - 0s 39ms/step - loss: 1.2024 - acc: 0.5437 - val_loss: 0.7052 - val_acc: 0.5714
Epoch 7/500
8/8 [==============================] - 0s 40ms/step - loss: 0.8630 - acc: 0.6577 - val_loss: 0.7061 - val_acc: 0.5238
Epoch 8/500
8/8 [==============================] - 0s 47ms/step - loss: 1.2697 - acc: 0.4

8/8 [==============================] - 0s 48ms/step - loss: 0.4996 - acc: 0.7845 - val_loss: 0.7124 - val_acc: 0.7143
Epoch 64/500
8/8 [==============================] - 0s 44ms/step - loss: 0.4120 - acc: 0.7845 - val_loss: 0.3511 - val_acc: 0.7619
Epoch 65/500
8/8 [==============================] - 0s 37ms/step - loss: 0.3539 - acc: 0.8732 - val_loss: 0.3447 - val_acc: 0.8571
Epoch 66/500
8/8 [==============================] - 0s 41ms/step - loss: 0.3636 - acc: 0.8606 - val_loss: 0.5553 - val_acc: 0.6667
Epoch 67/500
8/8 [==============================] - 0s 42ms/step - loss: 0.4215 - acc: 0.7352 - val_loss: 0.3278 - val_acc: 0.9048
Epoch 68/500
8/8 [==============================] - 0s 42ms/step - loss: 0.3596 - acc: 0.8986 - val_loss: 0.3302 - val_acc: 0.9048
Epoch 69/500
8/8 [==============================] - 0s 49ms/step - loss: 0.2898 - acc: 0.8225 - val_loss: 0.3452 - val_acc: 0.8095
Epoch 70/500
8/8 [==============================] - 0s 38ms/step - loss: 0.2477 - acc: 0.8732 - 

8/8 [==============================] - 0s 39ms/step - loss: 0.2629 - acc: 0.9113 - val_loss: 0.2783 - val_acc: 0.9048
Epoch 126/500
8/8 [==============================] - 0s 34ms/step - loss: 0.1937 - acc: 0.9239 - val_loss: 0.3849 - val_acc: 0.8095
Epoch 127/500
8/8 [==============================] - 0s 38ms/step - loss: 0.2700 - acc: 0.8732 - val_loss: 0.2421 - val_acc: 0.9048
Epoch 128/500
8/8 [==============================] - 0s 36ms/step - loss: 0.1926 - acc: 0.9366 - val_loss: 0.2742 - val_acc: 0.9048
Epoch 129/500
8/8 [==============================] - 0s 39ms/step - loss: 0.2348 - acc: 0.9620 - val_loss: 0.2630 - val_acc: 0.9048
Epoch 130/500
8/8 [==============================] - 0s 36ms/step - loss: 0.2444 - acc: 0.8859 - val_loss: 0.4069 - val_acc: 0.8571
Epoch 131/500
8/8 [==============================] - 0s 46ms/step - loss: 0.5178 - acc: 0.7859 - val_loss: 0.4241 - val_acc: 0.7619
Epoch 132/500
8/8 [==============================] - 0s 45ms/step - loss: 0.6079 - acc: 0.

8/8 [==============================] - 0s 42ms/step - loss: 0.1675 - acc: 0.9239 - val_loss: 0.2394 - val_acc: 0.9524
Epoch 188/500
8/8 [==============================] - 0s 40ms/step - loss: 0.2034 - acc: 0.9366 - val_loss: 0.2535 - val_acc: 0.9524
Epoch 189/500
8/8 [==============================] - 0s 44ms/step - loss: 0.1741 - acc: 0.9239 - val_loss: 0.1671 - val_acc: 0.9048
Epoch 190/500
8/8 [==============================] - 0s 36ms/step - loss: 0.1160 - acc: 0.9620 - val_loss: 0.2491 - val_acc: 0.9048
Epoch 191/500
8/8 [==============================] - 0s 36ms/step - loss: 0.1981 - acc: 0.9239 - val_loss: 0.1994 - val_acc: 0.9048
Epoch 192/500
8/8 [==============================] - 0s 40ms/step - loss: 0.1546 - acc: 0.9493 - val_loss: 0.1860 - val_acc: 0.9048
Epoch 193/500
8/8 [==============================] - 0s 36ms/step - loss: 0.1174 - acc: 0.9620 - val_loss: 0.1983 - val_acc: 0.9524
Epoch 194/500
8/8 [==============================] - 0s 40ms/step - loss: 0.1408 - acc: 0.

8/8 [==============================] - 0s 41ms/step - loss: 0.2231 - acc: 0.9239 - val_loss: 0.0879 - val_acc: 1.0000
Epoch 250/500
8/8 [==============================] - 0s 36ms/step - loss: 0.2343 - acc: 0.8732 - val_loss: 0.1304 - val_acc: 0.9524
Epoch 251/500
8/8 [==============================] - 0s 40ms/step - loss: 0.1197 - acc: 0.9620 - val_loss: 0.1998 - val_acc: 0.9048
Epoch 252/500
8/8 [==============================] - 0s 35ms/step - loss: 0.0687 - acc: 0.9746 - val_loss: 0.1584 - val_acc: 0.9524
Epoch 253/500
8/8 [==============================] - 0s 42ms/step - loss: 0.1462 - acc: 0.9746 - val_loss: 0.1230 - val_acc: 0.9524
Epoch 254/500
8/8 [==============================] - 0s 39ms/step - loss: 0.1270 - acc: 0.9620 - val_loss: 0.1472 - val_acc: 0.9048
Epoch 255/500
8/8 [==============================] - 0s 38ms/step - loss: 0.1279 - acc: 0.9493 - val_loss: 0.2760 - val_acc: 0.9048
Epoch 256/500
8/8 [==============================] - 0s 35ms/step - loss: 0.0836 - acc: 0.

8/8 [==============================] - 0s 36ms/step - loss: 0.1073 - acc: 0.9493 - val_loss: 0.1402 - val_acc: 0.9524
Epoch 312/500
8/8 [==============================] - 0s 39ms/step - loss: 0.0843 - acc: 0.9620 - val_loss: 0.1310 - val_acc: 0.9524
Epoch 313/500
8/8 [==============================] - 0s 40ms/step - loss: 0.0411 - acc: 1.0000 - val_loss: 0.1528 - val_acc: 0.9524
Epoch 314/500
8/8 [==============================] - 0s 34ms/step - loss: 0.1062 - acc: 0.9746 - val_loss: 0.2178 - val_acc: 0.8571
Epoch 315/500
8/8 [==============================] - 0s 37ms/step - loss: 0.0598 - acc: 0.9746 - val_loss: 0.1014 - val_acc: 0.9524
Epoch 316/500
8/8 [==============================] - 0s 38ms/step - loss: 0.0766 - acc: 0.9620 - val_loss: 0.1618 - val_acc: 0.9524
Epoch 317/500
8/8 [==============================] - 0s 38ms/step - loss: 0.0636 - acc: 0.9746 - val_loss: 0.2629 - val_acc: 0.9524
Epoch 318/500
8/8 [==============================] - 0s 37ms/step - loss: 0.0870 - acc: 0.

8/8 [==============================] - 0s 39ms/step - loss: 0.0797 - acc: 0.9620 - val_loss: 0.2638 - val_acc: 0.9048
Epoch 374/500
8/8 [==============================] - 0s 38ms/step - loss: 0.0318 - acc: 0.9873 - val_loss: 0.5143 - val_acc: 0.8571
Epoch 375/500
8/8 [==============================] - 0s 38ms/step - loss: 0.0581 - acc: 0.9620 - val_loss: 0.3420 - val_acc: 0.8571
Epoch 376/500
8/8 [==============================] - 0s 44ms/step - loss: 0.0769 - acc: 0.9746 - val_loss: 0.4581 - val_acc: 0.8571
Epoch 377/500
8/8 [==============================] - 0s 47ms/step - loss: 0.0352 - acc: 0.9746 - val_loss: 0.4494 - val_acc: 0.8571
Epoch 378/500
8/8 [==============================] - 0s 35ms/step - loss: 0.0805 - acc: 0.9873 - val_loss: 0.1224 - val_acc: 0.9524
Epoch 379/500
8/8 [==============================] - 0s 39ms/step - loss: 0.0433 - acc: 0.9746 - val_loss: 0.7704 - val_acc: 0.8571
Epoch 380/500
8/8 [==============================] - 0s 37ms/step - loss: 0.0924 - acc: 0.

8/8 [==============================] - 0s 38ms/step - loss: 0.0405 - acc: 0.9873 - val_loss: 0.1739 - val_acc: 0.9048
Epoch 436/500
8/8 [==============================] - 0s 36ms/step - loss: 0.0205 - acc: 1.0000 - val_loss: 0.1875 - val_acc: 0.9048
Epoch 437/500
8/8 [==============================] - 0s 38ms/step - loss: 0.0655 - acc: 0.9620 - val_loss: 0.3136 - val_acc: 0.8571
Epoch 438/500
8/8 [==============================] - 0s 38ms/step - loss: 0.0479 - acc: 1.0000 - val_loss: 0.1551 - val_acc: 0.9048
Epoch 439/500
8/8 [==============================] - 0s 37ms/step - loss: 0.0321 - acc: 0.9873 - val_loss: 0.2404 - val_acc: 0.8571
Epoch 440/500
8/8 [==============================] - 0s 38ms/step - loss: 0.0545 - acc: 0.9746 - val_loss: 0.1522 - val_acc: 0.9048
Epoch 441/500
8/8 [==============================] - 0s 38ms/step - loss: 0.0534 - acc: 0.9746 - val_loss: 0.1392 - val_acc: 0.9524
Epoch 442/500
8/8 [==============================] - 0s 39ms/step - loss: 0.0847 - acc: 0.

8/8 [==============================] - 0s 40ms/step - loss: 0.0234 - acc: 1.0000 - val_loss: 0.2601 - val_acc: 0.8571
Epoch 498/500
8/8 [==============================] - 0s 43ms/step - loss: 0.0268 - acc: 0.9873 - val_loss: 0.1141 - val_acc: 0.9048
Epoch 499/500
8/8 [==============================] - 0s 52ms/step - loss: 0.0097 - acc: 1.0000 - val_loss: 0.1187 - val_acc: 0.9048
Epoch 500/500
8/8 [==============================] - 0s 52ms/step - loss: 0.0732 - acc: 0.9746 - val_loss: 0.1265 - val_acc: 0.9048


In [48]:
print(use_model('3.jpg'))

[1]
